# Use Case 3: User interface to research intra-anaesthesisa hypotension
<span style = "font-size:22px">This notebook illustrates the usage of the vitabel package to visualize, annotate and process time-series data from the medical field. Please find the detailed, searchable documentation here: 
[![Documentation Status](https://readthedocs.org/projects/vitabel/badge/?version=latest)](https://vitabel.readthedocs.io/en/latest/?badge=latest)<br>
In this case we analyze non-waveform data from an anesthesia chart and add further labels to this data. This notebook in particular demonstrates how `vitabel` can be used outside of resuscitation science, in this case to study **intra-operative hypotension**. In especially the plotting will be wrapped into an overarching user interface to **validate** the data. </span>

In [16]:
from vitabel import Vitals, Label
import numpy as np
import pandas as pd

## 1) Load data

<span style = "font-size:18px">A vitabel object is initialized and data which was saved with vitabel before is loaded again.

In [2]:
case = Vitals()
case.load_data("data/usecase_3.json")

<span style = "font-size:18px">We get an overview over all channels and labels in the signal

In [3]:
case.info()

,Name,Length,First Entry,Last Entry,Offset
0,MAP,300,2024-12-24 12:24:24,2024-12-24 14:53:54,0 days 00:00:00
1,DIA,300,2024-12-24 12:24:24,2024-12-24 14:53:54,0 days 00:00:00
2,SYS,300,2024-12-24 12:24:24,2024-12-24 14:53:54,0 days 00:00:00


,Name,Length,First Entry,Last Entry,Offset
0,Remifentanil,1,2024-12-24 12:42:01.507886041,2024-12-24 14:47:10.070170558,0 days 00:00:00
1,Sevofluran,1,2024-12-24 12:37:32.863636363,2024-12-24 14:48:20.136363636,0 days 00:00:00
2,Event,8,2024-12-24 12:17:00,2024-12-24 14:55:11,0 days 00:00:00
3,Medication,14,2024-12-24 12:31:32.409090909,2024-12-24 14:36:55.743100799,0 days 00:00:00


## 2) Process Data
<span style = "font-size:18px">In this project, we're approaching things a little differently. Our primary focus is on the mean arterial pressure (MAP) during anaesthesia. To analyze this accurately, we need to validate both the MAP recordings
 and the defined anaesthesia interval.</span>

<span style = "font-size:18px">Conceptually `vitabel` devides time-series data into `channels` and `labels`:<br>
 - <span style = "font-size:18px">**channels** contain raw data recorded by a device
 - <span style = "font-size:18px">**labels** provide additional information—either annotated manually or 
derived from channel data<br>

<span style = "font-size:18px">This distinction can become blurred in cases where the recording device itself generates derived values. For example, if a monitor derives end-tidal CO₂ from a capnography waveform, one could argue whether these values should be considered part of a channel or stored as a label.<br>
However in the present use case, to actually remove and add MAP values in an interactive plot we have to **convert** the MAP recordings from the `channel` into a `label`.
</span>


In [4]:
map_channel = case.get_channel(name='MAP')
map_label = Label.from_dict(map_channel.to_dict())
map_label.plotstyle.update({"lw":0.8,"alpha":0.8, "c":"#393b41", "ls":"-", "ms":10})

case.get_channel("MAP").attach_label(map_label) 

<span style = "font-size:18px">For analysis we are intereseted in the timespan between _Induction_ and _Anaesthesia End_.<br>
We therefore try to extract the timepoints of interest from the cahnnel _Events_ and generate a **new Label** called _Analsysis_ with them. As we want to manipulate both timepoints independently from each other, we deliberately do not use an normal `Label` and not an `IntervalLabel`. The timespan we are going to analyse will be defined by the extremes of the `time_index` of the label _Analysis_.

In [5]:
t_index, data, text_data  = case.get_label("Event").get_data()
mask = np.isin(text_data, ["Induction", "Anaesthesia End"])
analysis_label=Label(
    name="Analysis",
    time_index=t_index[mask], 
    data=None, 
    text_data=None, 
    plotstyle={"color" : "crimson", "lw" : 3, "alpha" : 0.5}, 
    plot_type="vline",
    vline_text_source="disabled"
)
case.add_global_label(analysis_label)

## 3) Interactively plot and label data

<span style = "font-size:18px">As in the previous use cases we initialize our plot with `plot_interactive`. We than adapt the figure more extensively by editing the `figure` and its `axes` directly.

In [55]:
t_analysis, *_ = case.get_label("Event").get_data()
padding_time = 2
plot_start=t_analysis.min() - pd.to_timedelta(padding_time, "m") # adds 2 minutes before first event
plot_stop=t_analysis.max() + pd.to_timedelta(padding_time, "m") # adds 2 minutes after the last event

plot = case.plot_interactive(
    channels = [[0, 1, 2], [], []],
    labels = [["Event", "Analysis", "MAP"], ["Remifentanil", "Medication"], ["Sevofluran"]],
    time_unit= "m",
    start = plot_start,
    stop = plot_stop,
    subplots_kwargs = {"figsize": (16.5, 8), "gridspec_kw": {"height_ratios": [5, 1, 0.5]}},
)

fig = plot.center.figure
fig.suptitle("") #remove title
fig.subplots_adjust(hspace = 0)
axes = fig.get_axes()
axes[0].set_ylabel("Blood pressure (mmHg)")
axes[0].set_ylim(0)
axes[0].xaxis.set_ticks_position('top')
axes[1].set_xticks([])
for ax in axes[1:]:
    ax.set_yticks([])
    ax.set_xlabel("")
    ax.grid(False)

display(plot)

AppLayout(children=(Tab(children=(VBox(children=(HTML(value='\n                    <p>\n                    Ri…

<span style = "font-size:18px">To highlight episode of hypotension (i.e. MAP<65mmHg) we define a function to **highlight the area** in orange. </span><br>
_(will be visualized in the plot above)_

In [56]:
def show_auc(case, ax, cutoff : int=65):
    MAP=case.get_label('MAP')
    if MAP.is_time_absolute():
        reference_time = MAP.time_start - plot_start
        time_index = MAP.time_index + reference_time
    time_index /= pd.to_timedelta(1, unit="m")
    y2=np.array([cutoff]*len(MAP))
    ax.fill_between(time_index,MAP.data,y2,where=(MAP.data <= cutoff),interpolate=True, facecolor="#ff7f45", alpha=.8)

show_auc(case,axes[0], cutoff = 65)

In [59]:
MAP=case.get_label('MAP')
case.area_under_threshold(MAP, start_time=plot_start, stop_time=plot_stop, threshold=65)

AttributeError: 'str' object has no attribute 'name'

<span style = "font-size:18px">The data is saved again in a json file.

In [ ]:
def on_draw(event, case=case):
    global axes, endpoints, widget_hbox




    # check wether legend was redrawn
    if axes[0].get_legend():
        for ax in axes[:-1]:  
            ax.grid(False) #Remove grid
            ax.set_xlabel("") # Remove the x-axis label      
            leg = ax.get_legend()
            if leg:
                leg.remove()  # Remove legend
                
        axes[0].grid(axis='y', visible=True)  # optional: keep y-axis grid

        for ax in axes[1:-2]:
            ax.set_yticks([])  
        
        # Align labels to the left so they line up with the right axis line
        for label in axes[2].get_yticklabels():
            label.set_horizontalalignment('right')

    # check if AUC is marked
    has_fill_between = any(isinstance(col, PolyCollection) for col in axes[0].collections)
    if not has_fill_between:
        show_auc(case, axes[0], cutoff=65)

    # calculate edpoints
    endpoints = calculate_endpoints(case)

    # Display Results
    widget_hbox["twa"].value = endpoints["twa"]
    widget_hbox["auc"].value = endpoints["auc"]
    widget_hbox["hypotens_dur"].value = endpoints["hypotens_dur"]
    widget_hbox["anae_dur"].value = endpoints["anae_dur"]
    widget_hbox["noa_equivalent"].value = endpoints["noa_equivalent"]
    widget_hbox["min_map"].value = endpoints["min_map"]
    widget_hbox["max_period_length_no_map"].value = endpoints["max_period_length_no_map"]
    widget_hbox["mean_period_length"].value = endpoints["mean_period_length"]


In [7]:
def _wrap_in_widget(case: Vitals, plot, data_out_path: Path, tobe_reviewed: List[str], case_index: int, container :widgets.VBox, case_holder: Dict[str, Vitals]):
    global endpoints, widget_hbox, message_output

    def save_callback(b):
        message_output.clear_output()
    
        with message_output:
            if pd.isna(endpoints["anae_dur"]):
                display(widgets.HTML(
                    "<span style='color:red; font-weight:bold; font-size:16px;'>"
                    "⚠️ Analysis duration is unclear. Please set Labels accordingly.</span><br><br>"
                ))
                return None
            
            # Normal behavior
            case.metadata.setdefault("spa_ga", {})
            case.metadata["spa_ga"]["remarks"]={"investigator":"ME",
                                                       "comment":remark_input.value, 
                                                       "date":str(datetime.now())
                                                      }
                                        
            if flag_check.value:
                case.metadata["spa_ga"]["flagged"]={"revision":True}
            else:
                case.metadata["spa_ga"]["flagged"]={"revision":False}
            
            case.metadata["spa_ga"]["anaesthesia_type_by_algorithm"] = text_case_type.value
            case.metadata["spa_ga"]["endpoints"]=endpoints
            
            case.save_data(data_out_path/ f"{case.metadata["case_id"]}.json")
            display(widgets.HTML(
                "<span style='color:green; font-weight:bold; font-size:16px;'>"
                "✅ Saved succesfully.</span><br><br>"
            ))
    
            # Remove case from todo list
            next_plot()


NameError: name 'Path' is not defined

In [ ]:
case.save_data("data/usecase_3_final.json")

In [ ]:
def plot_spaga(case: Vitals, data_out_path : Path, tobe_reviewed: List[str], case_index: int, container :widgets.VBox, case_holder: Dict[str, Vitals]):   
    global axes, endpoints, widget_hbox

    padding_time=4 # minutes before and after plot 

    # to keep adaptions alive over redraws
    def on_draw(event, case=case):
        global axes, endpoints, widget_hbox

        def show_auc(case, ax, cutoff : int=65):
            MAP=case.get_label('MAP')
            if MAP.is_time_absolute():
                reference_time = MAP.time_start - plot_start
                time_index = MAP.time_index + reference_time
            time_index /= pd.to_timedelta(1, unit="h")
            y2=np.array([cutoff]*len(MAP))
            ax.fill_between(time_index,MAP.data,y2,where=(MAP.data <= cutoff),interpolate=True, facecolor=orange, alpha=.5)


        # check wether legend was redrawn
        if axes[0].get_legend():
            for ax in axes[:-1]:  
                ax.grid(False) #Remove grid
                ax.set_xlabel("") # Remove the x-axis label      
                leg = ax.get_legend()
                if leg:
                    leg.remove()  # Remove legend
                    
            axes[0].grid(axis='y', visible=True)  # optional: keep y-axis grid

            for ax in axes[1:-2]:
                ax.set_yticks([])  
            
            # Align labels to the left so they line up with the right axis line
            for label in axes[2].get_yticklabels():
                label.set_horizontalalignment('right')

        # check if AUC is marked
        has_fill_between = any(isinstance(col, PolyCollection) for col in axes[0].collections)
        if not has_fill_between:
            show_auc(case, axes[0], cutoff=65)

        # calculate edpoints
        endpoints = calculate_endpoints(case)

        # Display Results
        widget_hbox["twa"].value = endpoints["twa"]
        widget_hbox["auc"].value = endpoints["auc"]
        widget_hbox["hypotens_dur"].value = endpoints["hypotens_dur"]
        widget_hbox["anae_dur"].value = endpoints["anae_dur"]
        widget_hbox["noa_equivalent"].value = endpoints["noa_equivalent"]
        widget_hbox["min_map"].value = endpoints["min_map"]
        widget_hbox["max_period_length_no_map"].value = endpoints["max_period_length_no_map"]
        widget_hbox["mean_period_length"].value = endpoints["mean_period_length"]

    t_analysis= case.get_label("Analysis").get_data()[0]
    plot_start=t_analysis.min() - pd.to_timedelta(padding_time, "m")
    plot_stop=t_analysis.max() + pd.to_timedelta(padding_time, "m")

    # Actual plotting
    plot= case.plot_interactive(channels=[["sys", "dia","map"],
                                          [],
                                          []],
                                labels=[['Anästhesie','Narkose','Chirurgie',"MAP","_dummy","Analysis",], 
                                        ['_localanaesthetics_bolus','_opioids_bolus', '_narcotics_bolus', '_relaxans_bolus', '_pressors_bolus', "_dummy", "Analysis"],
                                        ["_narcotics_cont", "_pressors_cont","_opioids_cont", "_dummy", "Analysis"]],     
                                channel_overviews=[["map"]],
                                subplots_kwargs={"figsize": (10, 5), "gridspec_kw": {"height_ratios": [8, 3, 2,1]}},
                                start=plot_start,
                                stop=plot_stop,
                                time_unit="h")
    fig = plot.center.figure
    axes = fig.get_axes()

    fig.canvas.mpl_connect('draw_event', lambda event: on_draw(event, case))
    fig.subplots_adjust(hspace=0)
    
    # Adapt plot
    axes[0].set_ylabel("Blood pressure (mmHg)")
    axes[0].xaxis.set_ticks_position('top')
    ymin,ymax=axes[0].get_ylim()
    axes[0].set_ylim(0,ymax)
    axes[3].set_ylim(0,ymax)

    text_labels = axes[0].findobj(lambda artist: isinstance(artist, Text) and hasattr(artist, "_from_vitals_label"))
    for artist in text_labels:
        artist.set_y(ymin + 0.1 * (ymax - ymin))

    axes[1].set_yticks([])  # Remove the y-axis label
    axes[1].set_ylim(0,1) # force to redraw subplot 1

    for ax in axes[1:-1]:
        ax.set_xticks([])  
    
    ymin,ymax=axes[2].get_ylim()
    ylength = ymax - ymin
    axes[2].set_ylim(ymin-ylength/4, ymax+ylength/4)
    
    # Move ticks and labels to the right side
    axes[2].yaxis.tick_right()
    axes[2].yaxis.set_label_position("right")
    # Move y-tick labels inside the plot
    axes[2].tick_params(axis='y', direction='in', pad=-5)  # Adjust pad to move inside

    return _wrap_in_widget(case=case, plot=plot, data_out_path=data_out_path, tobe_reviewed=tobe_reviewed, case_index=case_index, container=container, case_holder=case_holder)




In [9]:
case.get_label("Medication").to_dict()

{'name': 'Medication',
 'time_index': array([   0.        ,  112.27272727,  165.45454545,  922.10702808,
        1429.55926596, 2255.19777191, 2888.82732299, 3288.38881479,
        4276.32060403, 5451.68970636, 5800.50597437, 6053.77492914,
        6560.31283865, 7682.87946444]),
 'data': None,
 'text_data': array(['Fentanyl', 'Propofol', 'Esmeron', 'Phenylephrin', 'Phenylephrin',
        'Phenylephrin', 'Phenylephrin', 'Phenylephrin', 'Phenylephrin',
        'Phenylephrin', 'Phenylephrin', 'Phenylephrin', 'Phenylephrin',
        'Piritramid'], dtype=object),
 'time_start': '2024-12-24 12:28:52.863636363',
 'time_unit': 's',
 'offset': 0.0,
 'is_interval': False,
 'plotstyle': {'linestyle': '--', 'marker': None, 'color': 'blue'},
 'metadata': {},
 'plot_type': 'combined',
 'vline_text_source': 'text_data'}